In [1]:
#! /usr/bin/env python
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

# Parameters
# ==================================================

# Data loading params change (2->5)
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
#tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
#tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")
tf.flags.DEFINE_string("class1", "./data/rt-polaritydata/review1.txt", "Data source for the class1 data.")
tf.flags.DEFINE_string("class2", "./data/rt-polaritydata/review2.txt", "Data source for the class2 data.")
tf.flags.DEFINE_string("class3", "./data/rt-polaritydata/review3.txt", "Data source for the class3 data.")
tf.flags.DEFINE_string("class4", "./data/rt-polaritydata/review4.txt", "Data source for the class4 data.")
tf.flags.DEFINE_string("class5", "./data/rt-polaritydata/review5.txt", "Data source for the class5 data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters 
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
FLAGS = tf.flags.FLAGS
# FLAGS._parse_flags()
# print("\nParameters:")
# for attr, value in sorted(FLAGS.__flags.items()):
#     print("{}={}".format(attr.upper(), value))
# print("")

def preprocess():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    #x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file) #Maybe..more Flags..
    x_text, y = data_helpers.load_data_and_labels(FLAGS.class1, FLAGS.class2,FLAGS.class3,FLAGS.class4,FLAGS.class5) 
    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev

def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1], 
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding_dim,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = data_helpers.batch_iter(
                list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

def main(argv=None):
    x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()
    train(x_train, y_train, vocab_processor, x_dev, y_dev)
    print("OVER")
if __name__ == '__main__':
    tf.app.run()

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading data...
-------x_text_length:  10000
Vocabulary Size: 10539
Train/Dev split: 9000/1000
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name

2018-07-18T13:28:31.787078: step 91, loss 3.10176, acc 0.34375
2018-07-18T13:28:43.756066: step 92, loss 3.13486, acc 0.21875
2018-07-18T13:28:54.592083: step 93, loss 2.89378, acc 0.15625
2018-07-18T13:29:05.363274: step 94, loss 2.54536, acc 0.34375
2018-07-18T13:29:15.204950: step 95, loss 2.73049, acc 0.3125
2018-07-18T13:29:25.025683: step 96, loss 2.37664, acc 0.375
2018-07-18T13:29:35.103728: step 97, loss 3.02841, acc 0.328125
2018-07-18T13:29:45.051123: step 98, loss 2.85416, acc 0.3125
2018-07-18T13:29:55.241865: step 99, loss 2.22144, acc 0.375
2018-07-18T13:30:04.690594: step 100, loss 2.72093, acc 0.296875

Evaluation:
2018-07-18T13:31:53.811655: step 100, loss 1.40876, acc 0.421

Saved model checkpoint to C:\Users\dkdk6\cnn-text-classification-tf-master\runs\1531944623\checkpoints\model-100

2018-07-18T13:32:05.209172: step 101, loss 2.93039, acc 0.359375
2018-07-18T13:32:15.925508: step 102, loss 2.27612, acc 0.390625
2018-07-18T13:32:26.538123: step 103, loss 3.58479, a

2018-07-18T13:54:21.503506: step 214, loss 1.91329, acc 0.4375
2018-07-18T13:54:31.679289: step 215, loss 2.1654, acc 0.359375
2018-07-18T13:54:42.876341: step 216, loss 1.69333, acc 0.5
2018-07-18T13:54:53.492946: step 217, loss 2.4624, acc 0.3125
2018-07-18T13:55:03.336616: step 218, loss 1.95449, acc 0.421875
2018-07-18T13:55:14.134735: step 219, loss 2.28733, acc 0.359375
2018-07-18T13:55:24.476076: step 220, loss 1.90173, acc 0.28125
2018-07-18T13:55:34.114297: step 221, loss 1.58013, acc 0.453125
2018-07-18T13:55:44.046731: step 222, loss 1.97734, acc 0.421875
2018-07-18T13:55:54.269389: step 223, loss 1.98063, acc 0.34375
2018-07-18T13:56:04.473098: step 224, loss 2.15563, acc 0.375
2018-07-18T13:56:14.202076: step 225, loss 1.70875, acc 0.4375
2018-07-18T13:56:24.095615: step 226, loss 1.6982, acc 0.40625
2018-07-18T13:56:35.412360: step 227, loss 1.81428, acc 0.40625
2018-07-18T13:56:46.277286: step 228, loss 2.03463, acc 0.359375
2018-07-18T13:56:57.040498: step 229, loss 1.9

2018-07-18T14:17:45.023008: step 340, loss 1.65063, acc 0.4375
2018-07-18T14:17:55.295532: step 341, loss 1.28523, acc 0.515625
2018-07-18T14:18:05.531157: step 342, loss 1.21802, acc 0.5625
2018-07-18T14:18:14.226899: step 343, loss 1.54785, acc 0.4375
2018-07-18T14:18:22.810938: step 344, loss 1.69342, acc 0.390625
2018-07-18T14:18:31.385006: step 345, loss 1.31445, acc 0.5625
2018-07-18T14:18:39.846375: step 346, loss 1.42363, acc 0.4375
2018-07-18T14:18:48.323700: step 347, loss 1.41059, acc 0.46875
2018-07-18T14:18:56.732211: step 348, loss 1.41576, acc 0.546875
2018-07-18T14:19:05.383073: step 349, loss 1.41621, acc 0.453125
2018-07-18T14:19:13.902287: step 350, loss 1.58809, acc 0.40625
2018-07-18T14:19:22.443443: step 351, loss 1.4225, acc 0.453125
2018-07-18T14:19:31.069371: step 352, loss 1.49878, acc 0.5
2018-07-18T14:19:39.705273: step 353, loss 1.27601, acc 0.546875
2018-07-18T14:19:48.385058: step 354, loss 1.35405, acc 0.609375
2018-07-18T14:19:57.416900: step 355, loss 

...

2018-07-18T16:36:31.811743: step 1084, loss 0.832451, acc 0.703125
2018-07-18T16:36:40.451632: step 1085, loss 0.675539, acc 0.75
2018-07-18T16:36:50.759063: step 1086, loss 0.680009, acc 0.765625
2018-07-18T16:36:59.579472: step 1087, loss 0.726575, acc 0.75
2018-07-18T16:37:08.316104: step 1088, loss 0.858564, acc 0.671875
2018-07-18T16:37:17.342962: step 1089, loss 0.809587, acc 0.703125
2018-07-18T16:37:27.981506: step 1090, loss 0.838878, acc 0.71875
2018-07-18T16:37:38.920250: step 1091, loss 0.74223, acc 0.75
2018-07-18T16:37:48.235334: step 1092, loss 0.774217, acc 0.734375
2018-07-18T16:37:58.345293: step 1093, loss 0.854227, acc 0.734375
2018-07-18T16:38:08.991818: step 1094, loss 0.717081, acc 0.703125
2018-07-18T16:38:20.331488: step 1095, loss 0.859103, acc 0.65625
2018-07-18T16:38:29.564793: step 1096, loss 0.807482, acc 0.734375
2018-07-18T16:38:39.216977: step 1097, loss 0.935231, acc 0.65625
2018-07-18T16:38:50.257985: step 1098, loss 0.69847, acc 0.765625
2018-07-18T1

2018-07-18T16:58:33.933685: step 1203, loss 0.685553, acc 0.765625
2018-07-18T16:58:42.357156: step 1204, loss 0.78852, acc 0.703125
2018-07-18T16:58:50.734747: step 1205, loss 0.908875, acc 0.6875
2018-07-18T16:58:59.330757: step 1206, loss 0.859608, acc 0.71875
2018-07-18T16:59:07.981619: step 1207, loss 0.884614, acc 0.671875
2018-07-18T16:59:16.659408: step 1208, loss 0.810946, acc 0.71875
2018-07-18T16:59:25.332211: step 1209, loss 0.671221, acc 0.78125
2018-07-18T16:59:34.411927: step 1210, loss 0.7518, acc 0.71875
2018-07-18T16:59:43.173492: step 1211, loss 0.707246, acc 0.75
2018-07-18T16:59:51.909128: step 1212, loss 0.52081, acc 0.8125
2018-07-18T17:00:01.094560: step 1213, loss 0.783322, acc 0.65625
2018-07-18T17:00:10.089501: step 1214, loss 0.739088, acc 0.71875
2018-07-18T17:00:20.241348: step 1215, loss 0.611592, acc 0.78125
2018-07-18T17:00:29.965340: step 1216, loss 0.48556, acc 0.859375
2018-07-18T17:00:39.788068: step 1217, loss 0.708775, acc 0.75
2018-07-18T17:00:50

2018-07-18T17:21:36.245744: step 1325, loss 0.607141, acc 0.828125
2018-07-18T17:21:46.090413: step 1326, loss 0.692707, acc 0.71875
2018-07-18T17:21:55.663807: step 1327, loss 0.671875, acc 0.71875
2018-07-18T17:22:05.668049: step 1328, loss 0.657797, acc 0.796875
2018-07-18T17:22:16.090173: step 1329, loss 0.521028, acc 0.765625
2018-07-18T17:22:26.316820: step 1330, loss 0.606835, acc 0.78125
2018-07-18T17:22:37.234620: step 1331, loss 0.578954, acc 0.78125
2018-07-18T17:22:48.427683: step 1332, loss 0.615783, acc 0.796875
2018-07-18T17:22:58.704196: step 1333, loss 0.618648, acc 0.765625
2018-07-18T17:23:09.306838: step 1334, loss 0.528538, acc 0.796875
2018-07-18T17:23:19.050775: step 1335, loss 0.601757, acc 0.796875
2018-07-18T17:23:28.326967: step 1336, loss 0.420476, acc 0.84375
2018-07-18T17:23:38.914648: step 1337, loss 0.523995, acc 0.875
2018-07-18T17:23:49.679854: step 1338, loss 0.405327, acc 0.84375
2018-07-18T17:24:03.239586: step 1339, loss 0.526424, acc 0.84375
2018-

SystemExit: 

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
print("Start the test")

Start the test
